In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import os

In [4]:
dataset = pd.read_csv('/content/drive/MyDrive/naive_bayes_dataset.csv')
dataset

,day,season,wind,rain,class
0,weekday,spring,none,none,ontime
1,weekday,winter,none,slight,ontime
2,weekday,winter,none,slight,ontime
3,weekday,winter,high,heavy,late
4,saturday,summer,normal,none,ontime
5,weekday,autumn,normal,none,very late
6,holiday,summer,high,slight,ontime
7,sunday,summer,normal,none,ontime
8,weekday,winter,high,heavy,very late
9,weekday,summer,none,slight,ontime


# **Prior Probability**

In [5]:
classes=list(dataset['class'].unique())

classesInstance=[]
for c in classes:
  classesInstance.append(dataset[dataset['class'] == c]['class'].count())

classProbability={}

n=sum(classesInstance)

for c in classesInstance:
  classIndex=classesInstance.index(c)
  classProbability[classes[classIndex]]=c/n

print(classes, '-', classesInstance, '-', classProbability)

['ontime', 'late', 'very late', 'cancelled'] - [14, 2, 3, 1] - {'ontime': 0.7, 'late': 0.1, 'very late': 0.15, 'cancelled': 0.05}


# **Conditional Probability**

In [6]:
columnList=dataset.columns.to_list()
attributes=columnList[:-1]
attributeDict={}
for a in attributes:
  attributeDict[a]=list(dataset[a].unique())
attributeValueDict={}

for key in attributeDict.keys():
  for value in attributeDict[key]:
    attributeValues=[]
    for c in classes:
      andInstance=len(dataset[(dataset[key] == value) & (dataset['class'] == c)])
      classInstance=len(dataset[(dataset['class'] == c)])
      computedProbability=andInstance/classInstance
      attributeValues.append(computedProbability)
      attributeValueDict[value]=attributeValues

posteriorDF=pd.DataFrame.from_dict(attributeValueDict,orient='index',columns=classes)
posteriorDF

,ontime,late,very late,cancelled
weekday,0.642857,0.5,1.000000,0.0
saturday,0.142857,0.5,0.000000,1.0
holiday,0.142857,0.0,0.000000,0.0
sunday,0.071429,0.0,0.000000,0.0
spring,0.285714,0.0,0.000000,1.0
winter,0.142857,1.0,0.666667,0.0
summer,0.428571,0.0,0.000000,0.0
autumn,0.142857,0.0,0.333333,0.0
none,0.357143,0.5,0.333333,0.0
high,0.285714,0.5,0.333333,1.0


# **Posterior Probability**

In [10]:
attributesInInterest=['weekday', 'spring', 'normal', 'slight']
classPosteriorProbability={i:0 for i in classes}
for c in classes:
  probabilityProductOfAttributes=1
  for attribute in attributesInInterest:
    probabilityProductOfAttributes*=posteriorDF.loc[attribute][c]
  productWithClassPriorProbability=classProbability[c]*probabilityProductOfAttributes
  classPosteriorProbability[c]=productWithClassPriorProbability
predictedClassPosteriorValue=max(classPosteriorProbability.values())
predictedClass=list(classPosteriorProbability.keys())[list(classPosteriorProbability.values()).index(predictedClassPosteriorValue)]
predictedClass

'ontime'

# **Normalization**

In [11]:
posteriorValues=classPosteriorProbability.values()
normalizedPosteriorValues=[value/sum(posteriorValues) for value in posteriorValues]
predictedClass=list(classPosteriorProbability.keys())[normalizedPosteriorValues.index(max(normalizedPosteriorValues))]
predictedClass

'ontime'